**Predict The Flight Ticket Price Hackathon**

**Import datasets and libraries**

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [37]:
train = pd.read_excel('Data_Train.xlsx')
test = pd.read_excel('Test_set.xlsx')

In [38]:
train.shape, test.shape

((10683, 11), (2671, 10))

In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [40]:
train.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [41]:
for i in train.columns:
    print("Unique values in", i, train[i].nunique())

Unique values in Airline 12
Unique values in Date_of_Journey 44
Unique values in Source 5
Unique values in Destination 6
Unique values in Route 128
Unique values in Dep_Time 222
Unique values in Arrival_Time 1343
Unique values in Duration 368
Unique values in Total_Stops 5
Unique values in Additional_Info 10
Unique values in Price 1870


In [42]:
for i in test.columns:
    print("Unique values in", i, test[i].nunique())

Unique values in Airline 11
Unique values in Date_of_Journey 44
Unique values in Source 5
Unique values in Destination 6
Unique values in Route 100
Unique values in Dep_Time 199
Unique values in Arrival_Time 704
Unique values in Duration 320
Unique values in Total_Stops 5
Unique values in Additional_Info 6


**Data Preprocessing/EDA**

In [44]:
# chechking the features in the Datasets

#Training Set

print("\nEDA on Training Set\n")
print("="*50)

print("\nFeatures/Columns : \n", train.columns)
print("\n\nNumber of Features/Columns : ", len(train.columns))
print("\nNumber of Rows : ",len(train))
print("\n\nData Types :\n", train.dtypes)

print("\n Contains NaN/Empty cells : ", train.isnull().values.any())

print("\n Total empty cells by column :\n", train.isnull().sum(), "\n\n")


EDA on Training Set


Features/Columns : 
 Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')


Number of Features/Columns :  11

Number of Rows :  10683


Data Types :
 Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

 Contains NaN/Empty cells :  True

 Total empty cells by column :
 Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64 




In [45]:

# Test Set

print("\nEDA on Test Set\n")
print("="*50)


print("\nFeatures/Columns : \n",test.columns)
print("\n\nNumber of Features/Columns : ",len(test.columns))
print("\nNumber of Rows : ",len(test))
print("\n\nData Types :\n", test.dtypes)
print("\n Contains NaN/Empty cells : ", test.isnull().values.any())
print("\n Total empty cells by column :\n", test.isnull().sum())


EDA on Test Set


Features/Columns : 
 Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info'],
      dtype='object')


Number of Features/Columns :  10

Number of Rows :  2671


Data Types :
 Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
dtype: object

 Contains NaN/Empty cells :  False

 Total empty cells by column :
 Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64


In [46]:
# Dealing with the Missing Value

print("Original Length of Training Set : ", len(train))

train = train.dropna()

print("Length of Training Set after dropping NaN: ", len(train))

Original Length of Training Set :  10683
Length of Training Set after dropping NaN:  10682


In [47]:
#Cleaning Journey Date 

#Training Set

train['Journey_Day'] = pd.to_datetime(train.Date_of_Journey, format='%d/%m/%Y').dt.day

train['Journey_Month'] = pd.to_datetime(train.Date_of_Journey, format='%d/%m/%Y').dt.month

# Test Set

test['Journey_Day'] = pd.to_datetime(test.Date_of_Journey, format='%d/%m/%Y').dt.day

test['Journey_Month'] = pd.to_datetime(test.Date_of_Journey, format='%d/%m/%Y').dt.month

# Compare the dates and delete the original date feature

train.drop(labels = 'Date_of_Journey', axis = 1, inplace = True)

test.drop(labels = 'Date_of_Journey', axis = 1, inplace = True)

In [48]:
# Cleaning Duration

# Training Set

duration = list(train['Duration'])

for i in range(len(duration)) :
    if len(duration[i].split()) != 2:
        if 'h' in duration[i] :
            duration[i] = duration[i].strip() + ' 0m'
        elif 'm' in duration[i] :
            duration[i] = '0h {}'.format(duration[i].strip())

dur_hours = []
dur_minutes = []  

for i in range(len(duration)) :
    dur_hours.append(int(duration[i].split()[0][:-1]))
    dur_minutes.append(int(duration[i].split()[1][:-1]))
    
train['Duration_hours'] = dur_hours
train['Duration_minutes'] =dur_minutes

train.drop(labels = 'Duration', axis = 1, inplace = True)

In [49]:

# Test Set

durationT = list(test['Duration'])

for i in range(len(durationT)) :
    if len(durationT[i].split()) != 2:
        if 'h' in durationT[i] :
            durationT[i] = durationT[i].strip() + ' 0m'
        elif 'm' in durationT[i] :
            durationT[i] = '0h {}'.format(durationT[i].strip())
            
dur_hours = []
dur_minutes = []  

for i in range(len(durationT)) :
    dur_hours.append(int(durationT[i].split()[0][:-1]))
    dur_minutes.append(int(durationT[i].split()[1][:-1]))
  
    
test['Duration_hours'] = dur_hours
test['Duration_minutes'] = dur_minutes

test.drop(labels = 'Duration', axis = 1, inplace = True)

In [50]:

#Cleaning Departure and Arrival Times

# Training Set


train['Depart_Time_Hour'] = pd.to_datetime(train.Dep_Time).dt.hour
train['Depart_Time_Minutes'] = pd.to_datetime(train.Dep_Time).dt.minute

train.drop(labels = 'Dep_Time', axis = 1, inplace = True)


train['Arr_Time_Hour'] = pd.to_datetime(train.Arrival_Time).dt.hour
train['Arr_Time_Minutes'] = pd.to_datetime(train.Arrival_Time).dt.minute

train.drop(labels = 'Arrival_Time', axis = 1, inplace = True)

In [51]:
# Test Set


test['Depart_Time_Hour'] = pd.to_datetime(test.Dep_Time).dt.hour
test['Depart_Time_Minutes'] = pd.to_datetime(test.Dep_Time).dt.minute


test.drop(labels = 'Dep_Time', axis = 1, inplace = True)

test['Arr_Time_Hour'] = pd.to_datetime(test.Arrival_Time).dt.hour
test['Arr_Time_Minutes'] = pd.to_datetime(test.Arrival_Time).dt.minute

test.drop(labels = 'Arrival_Time', axis = 1, inplace = True)

In [52]:
train.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Price,Journey_Day,Journey_Month,Duration_hours,Duration_minutes,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes
0,IndiGo,Banglore,New Delhi,BLR → DEL,non-stop,No info,3897,24,3,2,50,22,20,1,10
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2 stops,No info,7662,1,5,7,25,5,50,13,15
2,Jet Airways,Delhi,Cochin,DEL → LKO → BOM → COK,2 stops,No info,13882,9,6,19,0,9,25,4,25
3,IndiGo,Kolkata,Banglore,CCU → NAG → BLR,1 stop,No info,6218,12,5,5,25,18,5,23,30
4,IndiGo,Banglore,New Delhi,BLR → NAG → DEL,1 stop,No info,13302,1,3,4,45,16,50,21,35


In [53]:
test.head()

,Airline,Source,Destination,Route,Total_Stops,Additional_Info,Journey_Day,Journey_Month,Duration_hours,Duration_minutes,Depart_Time_Hour,Depart_Time_Minutes,Arr_Time_Hour,Arr_Time_Minutes
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,6,6,10,55,17,30,4,25
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,1 stop,No info,12,5,4,0,6,20,10,20
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,1 stop,In-flight meal not included,21,5,23,45,19,15,19,0
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,1 stop,No info,21,5,13,0,8,0,21,0
4,Air Asia,Banglore,Delhi,BLR → DEL,non-stop,No info,24,6,2,50,23,55,2,45


In [54]:
# Dependent Variable
Y_train = train.iloc[:,6].values  # 6 is the index of "Price" in the Training Set 

# Independent Variables
X_train = train.iloc[:,train.columns != 'Price'].values # selects all columns except "Price"

# Independent Variables for Test Set
X_test = test.iloc[:,:].values

In [55]:
from sklearn.preprocessing import LabelEncoder

le1 = LabelEncoder()
le2 = LabelEncoder()

In [56]:
# Training Set    

X_train[:,0] = le1.fit_transform(X_train[:,0])
X_train[:,1] = le1.fit_transform(X_train[:,1])
X_train[:,2] = le1.fit_transform(X_train[:,2])
X_train[:,3] = le1.fit_transform(X_train[:,3])
X_train[:,4] = le1.fit_transform(X_train[:,4])
X_train[:,5] = le1.fit_transform(X_train[:,5])

# Test Set

X_test[:,0] = le2.fit_transform(X_test[:,0])
X_test[:,1] = le2.fit_transform(X_test[:,1])
X_test[:,2] = le2.fit_transform(X_test[:,2])
X_test[:,3] = le2.fit_transform(X_test[:,3])
X_test[:,4] = le2.fit_transform(X_test[:,4])
X_test[:,5] = le2.fit_transform(X_test[:,5])

In [57]:
#Data After Encoding
print(pd.DataFrame(X_train).head())

  0  1  2    3  4  5   6  7   8   9   10  11  12  13
0  3  0  5   18  4  8  24  3   2  50  22  20   1  10
1  1  3  0   84  1  8   1  5   7  25   5  50  13  15
2  4  2  1  118  1  8   9  6  19   0   9  25   4  25
3  3  3  0   91  0  8  12  5   5  25  18   5  23  30
4  3  0  5   29  0  8   1  3   4  45  16  50  21  35


In [58]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


Y_train = Y_train.reshape((len(Y_train), 1)) 
Y_train = sc.fit_transform(Y_train)
Y_train = Y_train.ravel()

In [59]:
#After Scaling
print(pd.DataFrame(X_train).head()) # Xtrain after scaling

         0         1         2   ...        11        12        13
0 -0.410805 -1.658359  2.416534  ... -0.234950 -1.800436 -0.890014
1 -1.261152  0.890014 -0.973812  ...  1.363607 -0.050909 -0.587094
2  0.014369  0.040556 -0.295743  ...  0.031476 -1.363054  0.018745
3 -0.410805  0.890014 -0.973812  ... -1.034229  1.407030  0.321664
4 -0.410805 -1.658359  2.416534  ...  1.363607  1.115442  0.624584

[5 rows x 14 columns]


In [60]:

print(pd.DataFrame(Y_train).head())

          0
0 -1.125535
1 -0.309068
2  1.039783
3 -0.622209
4  0.914006


In [62]:
from sklearn.svm import SVR

svr = SVR(kernel = "rbf")
svr.fit(X_train,Y_train)
Y_pred = sc.inverse_transform(svr.predict(X_test))

In [63]:
print(pd.DataFrame(Y_pred).head())

              0
0  10547.002077
1   6773.635780
2  11482.732670
3  11281.247128
4   4379.175423


In [64]:
pd.DataFrame(Y_pred, columns = ['Price']).to_excel("Sub_Final_Predictions.xlsx", index = False)